# Nutrient optimization

## todo

* parallel GA with deap
* API
* set initial seed population with clusters
* GA: remove nutrients that are uncontrained when solving (eg, 1e10)
* random seed doesn't work

## Import custom helper functions

* def load_data():
* def do_clust(N,lim,req,nut):
* def evaluate(individual, nut,limt,reqd):
* def makeclusters(nclust,limt,reqd,nutrients ):
* def InitPopulation( pcls, ind_init,nfood, nclust, nseed,clust):
* def generate_ssdum(random, args):

In [1]:
import sys
print(sys.path)
from lib.libraries import *

['/home/pedwards/documents/SpartanSupper/diet_ga', '/home/pedwards/.conda/envs/spartansupper/lib/python37.zip', '/home/pedwards/.conda/envs/spartansupper/lib/python3.7', '/home/pedwards/.conda/envs/spartansupper/lib/python3.7/lib-dynload', '', '/home/pedwards/.conda/envs/spartansupper/lib/python3.7/site-packages', '/home/pedwards/.conda/envs/spartansupper/lib/python3.7/site-packages/IPython/extensions', '/home/pedwards/.ipython']


## Import external libraries

Most are standard, but we want the glpk solver for cvxopt, which requires the following,
```
$ sudo apt-get install libglpk-dev
$ sudo CVXOPT_BUILD_GLPK=1 pip install cvxopt
```

In [2]:
from os import path

import pickle
import pandas
import numpy
from deap import base, creator, tools, algorithms
from sklearn.preprocessing import normalize
from cvxopt import matrix, solvers # an alternative linear programming library
solvers.options['show_progress'] = False

from sklearn.cluster import KMeans
import random
from time import time

pandas.set_option('display.max_rows', None)
pandas.set_option('display.max_columns', None)

## Load food & nutrients from database

In [3]:
(nutrients,reqd,limt,food_desc,nutrient_desc)=load_data()
print( '[*] Loaded %d foods from database' % nutrients.shape[0] )

[*] Loaded 4492 foods from database


## Cluster

Observation: the optimization converges faster (and to lower minima) if provided a "seed" population with random baskets of *diverse* foods

Technique: I use a kmeans to find clusters of food types then sample from them (with a multinomial dist)

In [4]:
if not path.exists('clust.pkl'):
    Nclust=15
    clust=makeclusters(Nclust,limt,reqd,nutrients )
    pickle.dump( clust, open( "clust.pkl", "wb" ) )
else:
    clust=pickle.load( open( "clust.pkl", "rb" ) )
    print( '[*] Found pickle file with %d clusters and %d foods' % (clust.max()+1,len(clust)) )
    Nclust=clust.max()+1

[*] Found pickle file with 15 clusters and 4492 foods


## Genetic algorithm

This is the outermost optimization layer (the inner optim is in `evaluate()`)

Todo: play with early stopping. no need to keep going after improvement has ceased or slowed.

In [5]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin) # an individual comprises a list (of food IDs)
N_FOODS=6
Nseed=500
NT_DIM=nutrients.shape[0]
toolbox = base.Toolbox()
# Attribute generator 
toolbox.register("attr_foodid", random.randrange, NT_DIM)
# Structure initializers
toolbox.register("individual", tools.initRepeat, creator.Individual, 
    toolbox.attr_foodid, N_FOODS)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutUniformInt, low=0, up=NT_DIM, indpb=0.1)
#toolbox.register("select", tools.selBest, k=3)
toolbox.register("select", tools.selTournament, tournsize=10)
toolbox.register("evaluate", evaluate, nut=nutrients,limt=limt,reqd=reqd)

# used to make a seed population (only) ; per: https://deap.readthedocs.io/en/master/tutorials/basic/part1.html?highlight=seeding#seeding-a-population
toolbox.register("population_guess", InitPopulation, list, creator.Individual, N_FOODS,Nclust,Nseed,limt,reqd,nutrients )

stats = tools.Statistics(key=lambda ind: ind.fitness.values)
stats.register("min", numpy.min)
stats.register("median", numpy.median)
stats.register("max", numpy.max)

In [6]:
%%time

pop = toolbox.population(n=300) # totally random initial population
#pop = toolbox.population_guess()
pop, logbook = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=50, 
                                   stats=stats, verbose=True)

gen	nevals	median	std        	min   	max  
0  	300   	9e+09 	2.61809e+09	1698.8	9e+09
1  	167   	45516.9	4.4999e+09 	1401.96	9e+09
2  	180   	2469.8 	4.04099e+09	1278.49	9e+09
3  	169   	1698.8 	3.43248e+09	1278.49	9e+09
4  	190   	1401.96	1.89831e+09	1184.31	9e+09
5  	182   	1278.49	8.95489e+08	1113.66	9e+09
6  	186   	1230.1 	1.76363e+09	1113.66	9e+09
7  	161   	1184.31	1.9615e+09 	1113.66	9e+09
8  	181   	1113.66	1.76363e+09	980.189	9e+09
9  	161   	1113.66	1.89831e+09	980.189	9e+09
10 	160   	1067.24	1.26e+09   	912.503	9e+09
11 	172   	980.189	1.53528e+09	912.503	9e+09
12 	189   	980.189	2.19205e+09	912.503	9e+09
13 	190   	912.503	1.76363e+09	905.066	9e+09
14 	177   	912.503	1.9615e+09 	905.066	9e+09
15 	184   	912.503	1.69148e+09	887.459	9e+09
16 	170   	905.066	1.26e+09   	887.459	9e+09
17 	177   	905.066	1.35864e+09	887.459	9e+09
18 	179   	905.066	1.69148e+09	887.459	9e+09
19 	171   	887.459	1.89831e+09	887.459	9e+09
20 	185   	887.459	1.83246e+09	887.459	9e+09
21 	186   	887

In [7]:
best=tools.selBest(pop, k=1)
best=best[0]
evaluate(best, nut=nutrients,limt=limt,reqd=reqd)
nt=nutrients.iloc[best,:]
c = matrix(numpy.repeat(1.0,nt.shape[0]))
np_G= numpy.concatenate(
                        (   nt.transpose().values, 
                            nt.transpose().multiply(-1.0).values,
                            numpy.diag(numpy.repeat(-1,nt.shape[0])) 
                        )
                       ).astype(numpy.double) 
G = matrix( np_G ) 
h = matrix( numpy.concatenate( (
                limt.values, 
                reqd.multiply(-1.0).values, 
                numpy.repeat(0.0,nt.shape[0])
            ) ).astype(numpy.double) )    
o = solvers.lp(c, G, h, solver='glpk')
food_amounts=numpy.array(o['x'])[:,0]


## Print the best "diet"

This is the food and corresponding amount to eat (in grams, sorry bud!). The idea is if you eat all this in a day you have the nutrients you need for the day.

Todo: this would be better if split up into recipes. 

In [8]:
final_foods= [ best[i] for i in range(len(food_amounts)) if food_amounts[i]>1e-6 ] # select those with non-trivial amounts
final_food_amounts= food_amounts[ food_amounts>1e-6 ]

nt=nutrients.iloc[final_foods,:]
df1= nt.join(food_desc).loc[:,['long_desc']] #food_desc.iloc[final_foods]
df2=pandas.DataFrame(final_food_amounts*100,index=df1.index,columns=["amount"])
df3=pandas.DataFrame(nt.loc[:,208].values * df2.loc[:,'amount'].values/100 ,columns=['calories'], index=df2.index)
diet_table=df1.join(df2).join(df3)

In [9]:
diet_table

,long_desc,amount,calories
food_id,,,
10934,"Pork, cured, ham, shank, bone-in, separable le...",177.875826,222.344782
9307,"Rhubarb, raw",543.927658,114.224808
15012,"Fish, caviar, black and red, granular",49.500639,130.681688
11019,"Asparagus, frozen, cooked, boiled, drained, wi...",1030.127257,185.422906


In [10]:
nutrient_totals=pandas.DataFrame( ( 
                    numpy.dot( numpy.transpose(final_food_amounts), nt.values),
                    reqd,
                    limt
                  ), index=['Total','Amount required','Amount limit'], columns=nt.columns).transpose()

In [11]:
A=nt.join(df1).set_index('long_desc').transpose() * final_food_amounts
nutrient_report=A.join(nutrient_desc).join(nutrient_totals).set_index('name')
nutrient_report

,"Pork, cured, ham, shank, bone-in, separable lean only, unheated","Rhubarb, raw","Fish, caviar, black and red, granular","Asparagus, frozen, cooked, boiled, drained, without salt",Total,Amount required,Amount limit
name,,,,,,,
Protein,42.316662,4.895349,12.177157,30.388754,89.777921,53.459999,1.000000e+10
Total lipid (fat),5.674239,1.087855,8.860614,4.326534,19.949243,0.000000,1.000000e+10
"Carbohydrate, by difference",0.320177,24.694317,1.980026,19.778442,46.772960,0.000000,1.000000e+10
Ash,5.425213,4.133850,3.217541,6.283776,19.060381,0.000000,1.000000e+10
Energy,222.344788,114.224808,130.681686,185.422897,652.674184,0.000000,1.000000e+10
Starch,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000e+10
Sucrose,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000e+10
Glucose (dextrose),0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000e+10
Fructose,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000e+10
